In [41]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
pip install transformers


In [42]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TerraDataset(Dataset):


    def __init__(self, file):
        self.file = file
        self.strings = self.load()[0]
        self.predict = torch.tensor(self.load()[1])
        self.tokens = self.transform()

    def load(self):
      data = [json.loads(d) for d in open(self.file, encoding="utf-8")]
      s1 = []
      s2 = []
      for a in data:
        s1.append( a['premise']+' ~> '+a['hypothesis'])
        if a['label']== 'entailment':
          s2.append(1)
        else:
          s2.append(0)
      return [s1,s2]

    def transform(self):
      model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
      tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
      input_ids = []
      for string in self.strings:
        input_ids.append(tokenizer(string, return_length=True))
      return input_ids


    def __len__(self):
        return len(self.strings)


    def __getitem__(self, idx):
        return self.tokens[idx]

In [43]:
from transformers import DataCollatorForLanguageModeling
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm=False)

In [44]:
training_data = TerraDataset('/content/drive/MyDrive/Colab Notebooks/TERRa/train.jsonl')
predictions = training_data.predict
train_dataloader = DataLoader(training_data, batch_size = 8, shuffle = True, collate_fn = collator)

In [45]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [47]:

for batch_ndx, sample in enumerate(train_dataloader):
    a = model(input_ids = sample['input_ids'],attention_mask = sample['attention_mask'],return_dict = True,output_hidden_states = True)['hidden_states'][-1]
    l = sample['length']-1
    l = l.unsqueeze(-1)
    ind = l.repeat(1,768)
    ind = ind.unsqueeze(1)
    res = torch.gather(a, 1, ind)
    if batch_ndx == 0:
      break